Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# LAB9

Write a local-search algorithm (eg. an EA) able to solve the _Problem_ instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

- Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
- Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)


In [20]:
import random
from tqdm import tqdm
import lab9_lib

In [21]:
# fitness = lab9_lib.make_problem(1)
# for n in range(10):
#     ind = random.choices([0, 1], k=10)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)

The Python code in the lib defines an abstract class `AbstractProblem` and a function `make_problem(a)`.

The `AbstractProblem` class has the following methods and properties:

- `__init__`: Initializes the instance variable `_calls` to 0.
- `x`: An abstract property. Subclasses of `AbstractProblem` are expected to implement this.
- `calls`: A property that returns the number of times the instance has been called.
- `onemax`: A static method that takes a genome (a sequence of genes) and returns the sum of its genes, treating each gene as a boolean value.
- `__call__`: A special method that allows instances of the class to be called like functions. It increments `_calls`, computes fitnesses of the genome by slicing it into segments of length `x` and applying `onemax` to each segment, and then computes a value based on these fitnesses.

The `make_problem(a)` function defines a subclass of `AbstractProblem` with `x` implemented as a property that always returns `a`, and returns this subclass. This allows you to create problems with different values of `x` easily.

In summary, this code provides a framework for defining and working with optimization problems where the goal is to maximize the sum of genes in a genome, with some penalty for non-maximal segments. The `x` property determines the segment length for this computation. The `calls` property allows you to track how many times a problem instance has been called. The `make_problem(a)` function makes it easy to create problem instances with different segment lengths.

In [37]:
# solving with GA


class GA:
    def __init__(
        self,
        fitness_function,
        genome_length,
        population_size,
        tournament_size,
        mutation_probability,
        crossover_probability,
        max_iterations,
    ):
        self.fitness_function = fitness_function
        self.genome_length = genome_length
        self.population_size = population_size
        self.tournament_size = tournament_size
        self.mutation_probability = mutation_probability
        self.crossover_probability = crossover_probability
        self.max_iterations = max_iterations

    def initialize_population(self):
        return [
            random.choices([0, 1], k=self.genome_length)
            for _ in range(self.population_size)
        ]



    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_probability:
            crossover_point = random.randint(1, self.genome_length - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            return child1, child2
        else:
            return parent1, parent2

    def mutate(self, individual):
        mutated_child =   [1-bit if random.random() < self.mutation_probability else bit for bit in individual]
        return mutated_child

    def run(self):
        population = self.initialize_population()
        for gen in tqdm(range(self.max_iterations)):

            new_population = []
            fitness_values = [self.fitness_function(ind) for ind in population]
            parent1, parent2 = random.choices(
                population, weights=fitness_values, k=2
            )

            for _ in range(self.population_size // 2):
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.append(child1)
                new_population.append(child2)

            population = new_population
        return max(fitness_values)

GENOME_LENGTH = 100
POPULATION_SIZE = 100
TOURNAMENT_SIZE = 10
MUTATION_PROBABILITY = 0.01
CROSSOVER_PROBABILITY = 0.8
MAX_ITERATIONS = 10000

fitness_function = lab9_lib.make_problem(1)
genetic_algorithm = GA(fitness_function, GENOME_LENGTH, POPULATION_SIZE, TOURNAMENT_SIZE, MUTATION_PROBABILITY, CROSSOVER_PROBABILITY, MAX_ITERATIONS)

print(genetic_algorithm.run())

100%|██████████| 10000/10000 [00:25<00:00, 396.10it/s]

0.54
